<a href="https://colab.research.google.com/github/cosmos-dx/ALone/blob/main/Doctorg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
################################################################################################################################################################
###########################################################################################################################################################################
###########################################################################################################################################################################
################################################################################################################################################################
################################################################################################################################################################
################################################################################################################################################################

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

data = pd.read_csv('doctorg_data.csv')
pivot_table = data.pivot_table(values='weight', index='name', columns='symptom', fill_value=0)
X = pivot_table.values
y = pivot_table.index

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [2]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [20]:
def predict_disease(symptoms, pivot_table, model):
    input_vector = np.zeros(pivot_table.shape[1])

    for symptom in symptoms:
        if symptom in pivot_table.columns:
            symptom_index = pivot_table.columns.get_loc(symptom)
            input_vector[symptom_index] = pivot_table[symptom].max()

    input_vector = input_vector.reshape(1, -1)

    prediction = model.predict(input_vector)
    return prediction
#symptoms = ['Depressive or psychotic symptoms','Irregular heartbeat', 'Breathing fast' ,'Anxiety and nervousness','Palpitations', 'Dizziness','Chest tightness' ,'Insomnia', 'Abnormal involuntary movements']
symptoms = ['Diminished hearing', 'Fluid in ear', 'Redness in ear', 'Mass on ear', 'Plugged feeling in ear','Bleeding from ear','Skin growth', 'Pus draining from ear','Allergic reaction','Muscle swelling']
predicted_disease = predict_disease(symptoms, pivot_table, model)
print("Predicted Disease:", predicted_disease[0])


Predicted Disease: Cholesteatoma
